# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import yfinance as yf
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [ ]:
# Made my own list from wikipedia
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return [ticker.replace(".","-") for ticker in table['Symbol'].tolist()]
    
ticker_list = get_sp500_tickers()

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [ ]:
# I used Yahoo Finance instead of IEX Cloud API

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [4]:
def get_ticker_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        price = info.get('currentPrice')
        market_cap = info.get('marketCap')

        if price is None or market_cap is None:
            print(f"Skipping {ticker}: Missing price or market cap data")
            return None
            
        return {
            'Ticker': ticker,
            'Stock Price': price,
            'Market Capitalization' : round(market_cap / 1e9, 2),
            'Number of Shares to Buy': 'N/A'
        }

    except Exception as e:
        print(f"Error fetching {ticker}: {str(e)}")
        return None

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [5]:
ticker_list = get_sp500_tickers()
stocks = []

for ticker in ticker_list:
    result = get_ticker_data(ticker)
    if result:
        stocks.append(result)

HTTP Error 500: 


Skipping MDT: Missing price or market cap data


In [ ]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(stocks, columns=my_columns, index=range(1, len(stocks) + 1))

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
# N/A

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [8]:
portfolio_size = input('Enter the value of your portfolio:')
try:
    val = float(portfolio_size)
except ValueError:
    print("Please enter an integer")

Enter the value of your portfolio: 30000000


In [ ]:
position_size = val/len(final_dataframe.index)
for i in (final_dataframe.index):
    stock_price = final_dataframe.loc[i, 'Stock Price']
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/stock_price)
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
1,MMM,149.280,79.51,400
2,AOS,74.595,10.60,801
3,ABT,125.630,218.58,475
4,ABBV,190.565,336.61,313
5,ACN,280.665,174.81,212
...,...,...,...,...
498,XYL,131.210,31.93,455
499,YUM,146.490,40.72,407
500,ZBRA,333.570,16.96,179
501,ZBH,97.090,19.21,615


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [ ]:
filename = 'recommended_trades.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

dollar_format = writer.book.add_format({
        'num_format': '$#,##0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

integer_format = writer.book.add_format({
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [82]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format) 
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format) 
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format) 
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format) 

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [83]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1]) 

## Saving Our Excel Output

Saving our Excel file is very easy:

In [84]:
writer.close()